# The Delta Method and the NW Covariance Matrix


## Load Packages

In [1]:
using Compat, Missings        #in Julia 0.6 
#using Dates, DelimitedFiles  #in Julia 0.7

include("jlFiles/printmat.jl")

printlnPs (generic function with 2 methods)

# The Delta Method Applied to the Sharpe Ratio

In [2]:
x  = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)
x  = x[:,2]              #x is an excess return in % (on the US equity market)
T  = size(x,1)

388

## Point Estimates of Mean and 2nd Moment

Estimate the mean and second moment as usual 

$
\hat{\mu}   =\frac{1}{T}\sum_{t=1}^{T}x_{t}
$

$
\hat{\mu}_{2}  =\frac{1}{T}\sum_{t=1}^{T}x_{t}^{2}
$

Use GMM to estimate the covariance matrix ($V$) of $(\hat{\mu},\hat{\mu}_{2})$

In [3]:
μ   = mean(x)                  #estimates of the mean and 2nd moment
μ₂  = mean(x.^2)
printlnPs("\n","μ and μ₂",μ,μ₂)


μ and μ₂     0.602    21.505


The next cell constructs the variance-covariance matrix (called $V$) of the point estimates (collected into a vector $\hat{\beta}$). Recall that we typically have that

$
\sqrt{T}(\hat{\beta}-\beta_{0})\overset{d}{\rightarrow}N(0,V_{k\times k})
$

$V$ is constructed by using the usual GMM properties (discussed in detail later on). For now, you could just run the next cell without checking the details.

In [4]:
g    = [(x .- μ) (x.^2 .- μ₂)]    #moment conditions
T    = size(g,1)
gbar = vec(Compat.mean(g,dims=1))        #vector, 0.7 syntax
println("Sample moment conditions, gbar ")
printmat(gbar)

S = cov(g)                        #Var[sqrt(T)*gbar] = var(g) if iid 
#S = NWFn(g,1)                    #to use Newey-West instead

D = -Matrix(1.0I,2,2)             #-I(2)
V = inv(D*inv(S)*D')              #Var[sqrt(T)*(mu,μ₂)]
println("Cov(params)")
printmat(V/T)

Sample moment conditions, gbar 
    -0.000
     0.000

Cov(params)
     0.055    -0.138
    -0.138     4.643



## The Sharpe Ratio and Its Derivatives

The Sharpe ratio and its derivatives (with respect to the parameters of the
Sharpe ratio) are

$
\frac{\text{E}(x)}{\sigma(x)}  =\frac{\mu}{(\mu_{2}-\mu^{2})^{1/2}},\: \text{where}\: \beta=(\mu,\mu_{2})
$

Let $f(\beta)$ denote the Sharpe ratio where $\beta$ is a vector of parameters 
consisting of the mean and the second moment ($\mu,\mu_{2}$). The derivates are then

$
\frac{\partial f(\beta)}{\partial\beta^{\prime}}  = \left[
\begin{array}[c]{cc}
\frac{\mu_{2}}{(\mu_{2}-\mu^{2})^{3/2}} & \frac{-\mu}{2(\mu_{2}-\mu^{2})^{3/2}}
\end{array}
\right]  
$

For information on numerical derivatives, see the "integration" chapter of my *Julia Tutorial* (it contains also material on numerical derivatives).

In [5]:
"""
    SRFn(par)    

Calculate the Sharpe ratio from the mean and 2nd moment
"""
function SRFn(par)
  (μ,μ₂) = (par[1],par[2])          #E(Z),E(Z^2)
  s2 = μ₂ - μ^2
  SR = μ/sqrt(s2)
  df = hcat(μ₂/(μ₂ - μ^2)^(3/2), -μ/(2*(μ₂ - μ^2)^(3/2)))     #Jacobian of SR, 1x2  
  return SR, df
end

SRFn

In [6]:
(SR,df) = SRFn([μ,μ₂])  

printlnPs("Sharpe ratio from parameters and direct: ",SR,mean(x)/std(x))

println("\nDerivatives ")
printmat(df)

Sharpe ratio from parameters and direct:      0.131     0.131

Derivatives 
     0.221    -0.003



## Applying the Delta Method to the Sharpe Ratio


Recall that if

$
\sqrt{T}(\hat{\beta}-\beta_{0})\overset{d}{\rightarrow}N(0,V_{k\times k})  ,
$

then the distribution of the function $f(\hat{\beta})$ is

$
\sqrt{T}[f(\hat{\beta})-f(\beta_{0})] \overset{d}{\rightarrow}N(0,\Lambda_{q\times q})  
$

with 

$
\Lambda = \frac{\partial f(\beta_{0})}{\partial\beta^{\prime}} 
V 
\frac{\partial f(\beta_{0}) ^{\prime}}{\partial\beta}, \: 
\text{where } \: 
\frac{\partial f(\beta)}{\partial\beta^{\prime}} = 
\left[
\begin{array}[c]{ccc}
\frac{\partial f_{1}(\beta)}{\partial\beta_{1}} & \cdots &\frac{\partial f_{1}(\beta)}{\partial\beta_{k}} \\
\vdots & \ddots & \vdots \\
\frac{\partial f_{q}(\beta)  }{\partial\beta_{1}} & \cdots & \frac{\partial f_{q}(\beta)}{\partial\beta_{k}}
\end{array}
\right]  _{q\times k}
$

In [7]:
Std_SR = sqrt((df*V*df'/T)[1])            #[1] to convert from 1x1 matrix to scalar 

println("\nSR and its Std ")
printmat([SR Std_SR])

println("SR and 90% conf band: ")
printmat([SR (SR-1.65*Std_SR) (SR+1.65*Std_SR)])


SR and its Std 
     0.131     0.054

SR and 90% conf band: 
     0.131     0.042     0.220



# Newey-West

Let $g_t$ be a vector of data series (or moment conditions).

To calculate the Newey-West covariance matrix, we first need the
autocovariance matrices $\Omega_{s}=\text{Cov}(g_{t},g_{t-s})  $, 
which is estimated as 
$ 
\sum_{t=s+1}^{T} (g_{t}-\bar{g})(g_{t-s}-\bar{g})^{\prime}/T.
$

Then we form a linear
combination (with tent-shaped weights) of those autocovariance matrices (from
lag $-m$ to $m$), or equivalently

$
\text{Cov}(\sqrt{T}\bar{g})  = 
\Omega_{0} + \sum_{s=1}^{m}( 1-\frac{s}{m+1})  
(\Omega_{s}+\Omega_{s}^{\prime}).
$

The cells below illustrate this by estimating the std of a sample average in different ways. 

In [8]:
"""
    NWFn(g0,m=0)

Calculates covariance matrix of sqrt(T)*sample average.

# Usage
S = NWFn(g0,m)

# Input
- `g0::Array`: Txq array of q moment conditions
- `m:int`: scalar, number of lags to use

# Output
- `S::Array`: qxq covariance matrix

"""
function NWFn(g0,m=0)

  T = size(g0,1)                    #g0 is Txq
  m = min(m,T-1)                    #number of lags

  g = g0 .- Compat.mean(g0,dims=1)  #Normalizing to Eg=0, 0.7 syntax
  
  S = g'g/T                         #(qxT)*(Txq)
  for s = 1:m
    Gamma_s = g[s+1:T,:]'g[1:T-s,:]/T   #same as Sum[g(t)*g(t-s)',t=s+1,T]
    S       = S  +  ( 1 - s/(m+1) ) * (Gamma_s + Gamma_s')
  end

  return S

end

NWFn

In [9]:
T = 300                 #simulate two AR(1) process with different persistence
g = [randn(1,2)*10;zeros(T-1,2)]
for t = 2:T                        
    g[t,1] = 0.90*g[t-1,1] + randn()*10
    g[t,2] = 0.00*g[t-1,2] + randn()*10
end

println("                                                Series 1  Series 2")
printlnPs("Std(of average) according to an iid assumption:",sqrt.(var(g,1)/T))
printlnPs("Std(of average) according to NW:               ",sqrt.(diag(NWFn(g,15)/T)))

                                                Series 1  Series 2
Std(of average) according to an iid assumption:     1.479     0.550
Std(of average) according to NW:                    4.477     0.627
